Load Data

In [1]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
def upload(name):
  uploaded = drive.CreateFile({'title': name})
  uploaded.SetContentFile(name)
  uploaded.Upload()
  print('Uploaded file with ID {}'.format(uploaded.get('id')))

In [0]:
import pandas as pd
import numpy as np

In [0]:
PATH = './drive/My Drive/636/dataset/'
file_path = PATH + 'dataset_dropped.csv'

In [0]:
dataset = pd.read_csv(file_path)

In [7]:
dataset

,person_id,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,f_37,f_38,...,f_171,f_172,f_173,f_174,f_175,f_176,f_177,f_178,f_179,f_180,f_181,f_182,f_183,f_184,f_185,f_186,f_187,f_188,f_189,f_190,f_191,f_192,f_193,f_194,f_195,f_196,f_197,f_198,f_199,f_200,f_201,f_202,f_203,f_204,f_205,f_206,f_207,f_208,f_209,label
0,1,763.665,423.757,0.695357,769.403,477.315,0.721754,778.967,530.872,0.773077,790.443,580.605,0.721694,815.310,628.424,0.712524,834.437,672.418,0.780224,865.042,714.499,0.789111,907.123,745.103,0.803843,954.942,754.667,0.856622,1000.850,745.103,0.840945,1033.370,712.586,0.851339,1062.060,670.505,0.818585,1086.920,628.424,0.762580,...,943.465,681.982,0.877649,924.338,674.331,0.899704,897.559,657.116,0.863391,891.820,626.511,0.949315,924.338,634.162,0.878982,941.553,636.075,0.903428,960.680,634.162,0.791103,1002.760,628.424,0.906295,962.593,655.203,0.872532,941.553,655.203,0.923355,924.338,653.290,0.907176,865.042,456.274,0.887178,1029.540,467.751,0.889465,1
1,1,764.497,420.297,0.699779,766.441,476.678,0.735305,778.106,531.114,0.802437,787.827,579.718,0.752896,811.157,626.378,0.755506,830.598,671.094,0.793720,861.705,713.865,0.766783,906.421,743.028,0.787040,953.080,748.860,0.904945,999.740,739.139,0.799852,1030.850,708.033,0.809166,1060.010,669.150,0.815269,1085.280,624.434,0.792633,...,941.415,673.038,0.861085,923.918,671.094,0.952284,896.700,653.596,0.895886,890.867,624.434,1.002550,923.918,626.378,0.887870,939.471,632.211,0.863211,960.857,626.378,0.771206,1001.680,624.434,0.953828,962.801,653.596,0.846988,941.415,653.596,0.930170,923.918,651.652,0.883218,863.649,453.348,0.878478,1026.960,465.013,0.859063,1
2,1,760.839,422.602,0.698813,762.838,478.585,0.768472,774.835,528.570,0.857694,790.830,578.555,0.798236,808.824,626.540,0.819920,832.817,670.527,0.770925,864.807,706.516,0.739583,904.795,736.507,0.852870,952.781,748.503,0.890713,998.767,736.507,0.861623,1030.760,704.517,0.877598,1058.750,668.528,0.778922,1082.740,624.541,0.839065,...,944.783,672.527,0.874776,920.790,670.527,0.917472,898.797,654.532,0.869188,886.801,622.542,1.017780,920.790,624.541,0.916900,940.784,626.540,0.856891,964.777,624.541,0.837865,1000.770,622.542,0.965007,964.777,642.536,0.853121,942.784,648.534,0.835851,920.790,642.536,0.852623,858.809,450.594,0.848860,1022.760,460.591,0.856230,1
3,1,760.238,424.103,0.710734,764.239,476.111,0.777320,776.241,528.118,0.841734,792.243,580.125,0.793270,810.245,626.131,0.798826,838.249,670.137,0.780211,870.253,706.142,0.744071,906.258,734.146,0.833944,954.265,748.148,0.864520,1002.270,734.146,0.891102,1034.280,702.141,0.873774,1064.280,664.136,0.798164,1084.280,620.130,0.833952,...,952.265,670.137,0.921816,926.261,668.137,0.880207,902.258,652.135,0.899245,890.256,620.130,1.013840,926.261,622.131,0.854889,948.264,626.131,0.843004,968.267,622.131,0.875373,1004.270,620.130,0.931868,970.267,640.133,0.889719,950.264,646.134,0.863836,926.261,640.133,0.819194,860.252,452.107,0.836884,1024.270,456.108,0.846358,1
4,1,759.816,423.623,0.716206,763.816,477.617,0.767946,775.815,527.611,0.829785,791.813,579.605,0.788875,809.811,625.599,0.778499,839.807,669.594,0.795930,871.803,709.590,0.761219,911.799,735.587,0.779289,961.793,747.585,0.831127,1003.790,733.587,0.832449,1037.780,701.591,0.812543,1063.780,663.595,0.813895,1087.780,619.600,0.819781,...,953.794,669.594,0.940680,933.796,667.595,0.889411,903.800,651.596,0.909179,891.801,619.600,0.993394,933.796,623.600,0.862845,951.794,623.600,0.879477,969.792,621.600,0.894312,1011.790,619.600,0.912576,971.792,639.598,0.870145,951.794,647.597,0.890334,933.796,641.598,0.824826,863.804,451.620,0.823351,1029.780,455.619,0.863378,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

Some data cleaning

In [0]:
# dataset.loc[dataset.label == -1, "label"] = 0

In [0]:
# dataset.drop(dataset.iloc[:, 1:76], inplace = True, axis = 1)
# dataset.drop(dataset.iloc[:, 211:-1], inplace = True, axis = 1)

In [0]:
# dataset.to_csv('dataset_dropped.csv', index=False)
# upload('dataset_dropped.csv')

In [0]:
# dataset[dataset == 0.0] = np.nan
# dataset = dataset.drop(dataset.columns[dataset.apply(lambda col: col.isnull().sum() > 30)], axis=1)

In [0]:
# dataset = pd.concat([df_pos, df_neg], ignore_index=True)
# dataset.to_csv('dataset.csv', index=False)
# upload('dataset.csv')

In [0]:
# # continue to use the person_id of pos
# for i in range(85, 0, -1):
#   df_neg.loc[df_neg['person_id'] == i, 'person_id'] = i + 80
# df_neg.to_csv('neg.csv', index=False)
# upload('neg.csv')

Shuffle

In [0]:
# create a csv of person_id and label
people_id = [(i + 1) for i in range(0, 165)]
label = []
for i in range(0, 80):
  label.append(1)
for i in range(80, 165):
  label.append(0) 

In [0]:
# zip dataset and shuffle
idset = np.array((people_id, label)).T
np.random.shuffle(idset)

In [0]:
df_id = pd.DataFrame(idset, columns = ['people_id', 'label'])

In [0]:
from sklearn.model_selection import train_test_split
id_all = df_id['people_id']
label_all = df_id['label']
id_train, id_test, label_train, label_test = train_test_split(id_all, label_all, test_size=0.3, random_state=42)

In [84]:
print(id_train.shape, id_test.shape, label_train.shape, label_test.shape)

(115,) (50,) (115,) (50,)


In [0]:
# zipped = list(zip(people_id, label))
# data = pd.DataFrame(zipped, columns = ['people_id', 'label'])

Reshape Data into Segments and Prepare for Keras

In [0]:
# dataset['f_0'].values[dataset.index[dataset['person_id'] == 1].tolist()]

In [107]:
id_train[97]

99

In [0]:
df = dataset[dataset.columns[1:-1]]

In [145]:
df.shape

(6781, 210)

In [0]:
def create_segments(df, TIME_STAMP, idset):
  N_FEATURES = 210
  segments = []
  # get segment of each person_id
  for i in idset:
    # get every column of features of this person
    index_arr = df.index[df['person_id'] == i]
    df_ = df[df.columns[1:-1]]
    if len(index_arr) >= TIME_STAMP:
      feature_list = df_.values[df.index[index_arr[0] : index_arr[0] + TIME_STAMP]]     
    else:
      feature_list = df_.values[df.index[index_arr]]
      fillzero = np.zeros((TIME_STAMP - len(index_arr), N_FEATURES))
      feature_list = np.concatenate((feature_list, fillzero), axis=0)
    segments.append(feature_list)
  reshaped_segments = np.asarray(segments, dtype= np.float32)
  return reshaped_segments

In [0]:
# id_ = [1]
# x_ = create_segments(dataset, 30, id_)
# np.asarray(x_).shape

In [0]:
x_train = create_segments(dataset, 80, id_train)
x_test = create_segments(dataset, 80, id_test)

In [157]:
x_train.shape

(115, 80, 210)

In [158]:
x_test.shape

(50, 80, 210)

In [0]:
def create_labels(labelset):
  labels = []
  for i in labelset:
    labels.append(i)
  label_array = np.asarray(labels, dtype= np.float32)
  return label_array

In [0]:
y_train = create_labels(label_train)
y_test = create_labels(label_test)

Build model

In [0]:
from keras.layers.embeddings import Embedding
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [172]:
model = Sequential()
# now the model will take as input arrays of shape (*, *)
# and output arrays of shape (*, 32)
# after the first layer, you don't need to specify
# the size of the input anymore:
model.add(Dense(32, activation='relu'))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Train on 92 samples, validate on 23 samples
Epoch 1/10
92/92 [==============================] - 2s 20ms/step - loss: 0.6802 - acc: 0.6957 - val_loss: 0.6614 - val_acc: 0.4783
Epoch 2/10
92/92 [==============================] - 0s 2ms/step - loss: 0.6684 - acc: 0.6087 - val_loss: 0.5980 - val_acc: 0.6522
Epoch 3/10
92/92 [==============================] - 0s 2ms/step - loss: 0.6521 - acc: 0.6196 - val_loss: 0.6386 - val_acc: 0.6522
Epoch 4/10
92/92 [==============================] - 0s 2ms/step - loss: 0.6562 - acc: 0.6196 - val_loss: 0.5880 - val_acc: 0.6522
Epoch 5/10
92/92 [==============================] - 0s 2ms/step - loss: 0.6363 - acc: 0.6848 - val_loss: 0.5802 - val_acc: 0.7391
Epoch 6/10
92/92 [==============================] - 0s 2ms/step - loss: 0.6111 - acc: 0.7500 - val_loss: 0.5403 - val_acc: 0.6957
Epoch 7/10
92/92 [==============================] - 0s 2ms/step - loss: 0.5922 - acc: 0.7500 - val_loss: 0.5111 - val_acc: 0.8261
Epoch 8/10
92/92 [===========================

In [171]:
#Save the weights
filename='model_weights.hdf5'
upload(filename)
model.load_weights(filename)

FileNotFoundError: ignored

In [0]:
y_pred=model.predict(x_test)
# y_pred=np.argmax(y_pred,axis = 1)

In [177]:
y_pred

array([[0.680549  ],
       [0.6344054 ],
       [0.22135338],
       [0.6911482 ],
       [0.36277297],
       [0.12458891],
       [0.6358321 ],
       [0.4332291 ],
       [0.18771696],
       [0.43354958],
       [0.6899475 ],
       [0.43325278],
       [0.65502584],
       [0.29797867],
       [0.69346154],
       [0.12771735],
       [0.31072828],
       [0.20331752],
       [0.6344054 ],
       [0.22135338],
       [0.44434345],
       [0.45973048],
       [0.12427711],
       [0.49358314],
       [0.12754384],
       [0.44473824],
       [0.20331773],
       [0.4449192 ],
       [0.44434345],
       [0.5191289 ],
       [0.69454336],
       [0.4443434 ],
       [0.40144825],
       [0.12771744],
       [0.43405408],
       [0.6916594 ],
       [0.68699485],
       [0.4343499 ],
       [0.12420267],
       [0.4332292 ],
       [0.12359199],
       [0.22135338],
       [0.43370956],
       [0.4343499 ],
       [0.5556127 ],
       [0.22135338],
       [0.44434345],
       [0.127

In [174]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.5

K-fold Cross Validation

In [0]:
def create_model(classes):
    model = Sequential()
    model.add(Conv2D(32,kernel_size=(2,2),strides=(1,1),activation='relu',input_shape=(48,48,1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters=64,kernel_size=(2,2),strides=(1,1),activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2),strides=(1,1)))
    model.add(Dropout(0.25))#to prevent neural network from overfitting
    
    model.add(Conv2D(filters=128,kernel_size=(2,2),strides=(1,1),activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2),strides=(1,1)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters=256,kernel_size=(2,2),strides=(1,1),activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2),strides=(1,1)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    
    model.add(Dense(256,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    
    model.add(Dense(512,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    
    model.add(Dense(classes,activation='softmax'))
    
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [0]:
classes=2
model = create_convolutional_model(classes)
model.summary()

In [0]:
model.fit(X_train,y_train,batch_size=105,epochs=30,verbose=2)

Draw figure

In [0]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')

plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()